<a href="https://colab.research.google.com/github/Izuho/senior-thesis/blob/main/Qiskit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [ ]:
X1 = np.load('data/mnist_train_feat.npy')[0:3]
y1 = np.load('data/mnist_train_Label.npy')[0:3]
X2 = np.load('data/mnist_test_feat.npy')[0:2]
y2 = np.load('data/mnist_test_Label.npy')[0:2]
print(X1.shape, y1.shape)

In [ ]:
from qiskit.circuit import Parameter

def fraxis_gate(nx, ny, nz=None):
    nx, ny, nz = _validate(nx, ny, nz)
    theta = 2*np.arccos(nz)
    phi = 0
    if np.sin(theta/2)>1e-31:
        if nx/np.sin(theta/2)<=1:
            phi = np.arccos(nx/np.sin(theta/2)) if ny>=0 else np.pi-np.arccos(nx/np.sin(theta/2))
    circ = QuantumCircuit(1)
    circ.u(theta, phi, np.pi-phi,[0])
    return circ

def _validate(nx, ny, nz=None):
    if nz == None:
        nx = nx.real
        ny = ny.real
        nz2 = 1-nx**2.-ny**2.
        nz = np.sqrt(nz2).real if nz2>0 else 0
    return nx, ny, nz

In [ ]:
def FraxisFeatureMap(num_qubits, data):
    circ = QuantumCircuit(num_qubits)
    for i in range(
        data.shape[0]//2
    ):
        circ.compose(fraxis_gate(data[2*i], data[2*i+1]), qubits=[i%num_qubits], inplace=True)
        if (i+1)%num_qubits == 0:
            for j in range(0,num_qubits,2):
                if j+1 < num_qubits:
                    circ.cz(j,j+1)
            for j in range(1,num_qubits,2):
                if j+1 < num_qubits:
                    circ.cz(j,j+1)        
    return circ

In [ ]:
def FraxisAnsatz(num_qubits, params):
    circ = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        circ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    for j in range(0,num_qubits,2):
        if j+1 < num_qubits:
            circ.cz(j,j+1)
    for j in range(1,num_qubits,2):
        if j+1 < num_qubits:
            circ.cz(j,j+1)        
    return circ

In [ ]:
def replace_FraxisAnsatz(num_qubits, target, params):
    circX = QuantumCircuit(num_qubits)
    circY = QuantumCircuit(num_qubits)
    circZ = QuantumCircuit(num_qubits)
    circXY = QuantumCircuit(num_qubits)
    circXZ = QuantumCircuit(num_qubits)
    circYZ = QuantumCircuit(num_qubits)
    for i in range(target):
        circX.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circYZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    circX.x(target)
    circY.y(target)
    circZ.z(target)
    circXY.u(np.pi, np.pi*0.25, np.pi*0.75, target)
    circXZ.u(np.pi*0.5, 0, np.pi, target)
    circYZ.u(np.pi*0.5, np.pi*0.5, np.pi*0.5, target)
    for i in range(target+1, num_qubits, 1):
        circX.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circYZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    for j in range(0,num_qubits,2):
        if j+1 < num_qubits:
            circX.cz(j,j+1)
            circY.cz(j,j+1)
            circZ.cz(j,j+1)
            circXY.cz(j,j+1)
            circXZ.cz(j,j+1)
            circYZ.cz(j,j+1)
    for j in range(1,num_qubits,2):
        if j+1 < num_qubits:
            circX.cz(j,j+1)
            circY.cz(j,j+1)
            circZ.cz(j,j+1)
            circXY.cz(j,j+1)
            circXZ.cz(j,j+1)
            circYZ.cz(j,j+1)
    return [circX, circY, circZ, circXY, circXZ, circYZ]

In [ ]:
#from qiskit_ibm_runtime import Estimator, QiskitRuntimeService, Session
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp

class FraxClassify():
    def __init__(self, n_qubits, layer_size, world_size, service):
        self.n_qubits = n_qubits
        self.layer_size = layer_size
        self.params = np.zeros((layer_size, n_qubits, 2)) + 1/np.sqrt(3)
        #np.random.rand(layer_size, n_qubits, 2) / 2
        self.world_size =world_size
        self.service = service
    def fit_and_eval(self, X, y, X2, y2):
        observable = SparsePauliOp.from_list([("I"* (self.n_qubits-1)+"Z", 1)])
        '''
        with Session(
            #service=None, 
            backend="ibmq_qasm_simulator"
        ) as session:
        '''
        estimator = Estimator(
        #    session=session
        )
        for a in range(self.layer_size):
            for b in range(self.n_qubits):
                R = np.zeros((3,3))
                for c in range(y.shape[0]):
                    feature_map = FraxisFeatureMap(self.n_qubits, X[c])
                    qcs = []
                    for d in range(6):
                        qcs.append(QuantumCircuit(self.n_qubits))
                    for d in range(a):
                        for e in range(6):
                            qcs[e].compose(feature_map, inplace=True)
                            
                        original_ansatz = FraxisAnsatz(self.n_qubits, self.params[d])
                        for e in range(6):
                            qcs[e].compose(original_ansatz, inplace=True)

                    for d in range(6):
                        qcs[d].compose(feature_map, inplace=True)
                        
                    ansatzs = replace_FraxisAnsatz(self.n_qubits, b, self.params[a])
                    for d in range(6):
                        qcs[d].compose(ansatzs[d], inplace=True)
                                        
                    for d in range(a+1,self.layer_size,1):
                        for e in range(6):
                            qcs[e].compose(feature_map, inplace=True)
                            
                        original_ansatz = FraxisAnsatz(self.n_qubits, self.params[d])
                        for e in range(6):
                            qcs[e].compose(original_ansatz, inplace=True)

                    r6 = estimator.run(
                        circuits=qcs,
                        observables=[observable]*6,
                    ).result().values
                    
                    #print(r6)
                    
                    rx, ry, rz, rxy, rxz, ryz = r6[0], r6[1], r6[2], r6[3], r6[4], r6[5]
                    R[0,0] += y[c] * 2 * rx
                    R[0,1] += y[c] * (2 * rxy - rx - ry)
                    R[0,2] += y[c] * (2 * rxz - rx - rz)
                    R[1,1] += y[c] * 2 * ry
                    R[1,2] += y[c] * (2 * ryz - ry - rz)
                    R[2,2] += y[c] * 2 * rz                 
                R[1,0] = R[0,1]
                R[2,0] = R[0,2]
                R[2,1] = R[1,2]
                #print(R)
                eigenvalues, eigenvectors = np.linalg.eigh(R)
                self.params[a,b,:] = eigenvectors[0:2,np.argmax(eigenvalues)]
                print(np.max(eigenvalues))
                
        acc, score, acc2, score2 = 0,0,0,0
        for a in range(X.shape[0]):
            qc = QuantumCircuit(self.n_qubits)
            feature_map = FraxisFeatureMap(self.n_qubits, X[a])
            for b in range(self.layer_size):
                qc.compose(feature_map, inplace=True)
                qc.compose(FraxisAnsatz(self.n_qubits, self.params[b]), inplace=True)
            result = estimator.run(
                circuits=[qc], 
                observables=[observable]
            ).result().values
            if y[a] * result > 0:
                acc += 1
            score += y[a] * result
        for a in range(X2.shape[0]):
            qc = QuantumCircuit(self.n_qubits)
            feature_map = FraxisFeatureMap(self.n_qubits, X2[a])
            for b in range(self.layer_size):
                qc.compose(feature_map, inplace=True)
                qc.compose(FraxisAnsatz(self.n_qubits, self.params[b]), inplace=True)
            result = estimator.run(
                circuits=[qc], 
                observables=[observable]
            ).result().values
            if y2[a] * result > 0:
                acc2 += 1
            score2 += y2[a] * result
        print(acc, score*2, acc2, score2*2)

In [ ]:
model = FraxClassify(6, 2, None, None)
for i in range(5):
    model.fit_and_eval(X1, y1, X2, y2)